In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
import numpy
import math
import shap 
import tensorflow as tf
import matplotlib.pyplot as plt
import lightgbm as ltb

from functools import reduce
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectPercentile
from sklearn.metrics import confusion_matrix, classification_report, f1_score, auc, roc_curve, roc_auc_score, precision_score, recall_score, balanced_accuracy_score
from numpy.random import seed
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold, StratifiedKFold
seed(42)
tf.random.set_seed(38)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense

numpy.set_printoptions(threshold=sys.maxsize)


In [ ]:
# this function will return variable list that have correlation more than cut(0.8)

def find_correlated_features(df, threshold=0.8):
    corr_matrix = df.corr().abs()
    avg_corr = corr_matrix.mean(axis=1)
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    drop_columns = []

    for row in range(len(upper_triangle) - 1):
        col_idx = row + 1
        for col in range(col_idx, len(upper_triangle)):
            if upper_triangle.iloc[row, col] > threshold:
                if avg_corr.iloc[row] > avg_corr.iloc[col]:
                    drop_columns.append(row)
                else:
                    drop_columns.append(col)

    drop_columns = set(drop_columns)
    correlated_features = df.columns[list(drop_columns)]
    
    return correlated_features



In [ ]:
def get_missing_val_percentage(df):
    return (df.isnull().sum()* 100 / len(df))

def preprocessing(df, test_size, response_variable, thresh = 40, isms = False, skip_corr = False):
    keep = [ ]
    rem = []
    # Convert categorical and boolean varialbles to numeric using LabelEncoder
#     cat_cols = df.select_dtypes(include=['bool', 'object']).columns
#     for col in cat_cols:
#         labelencoder = LabelEncoder()
#         df[col] = labelencoder.fit_transform(df[col])
        
    # remove rows with missing 'response variable'
#     df = df.dropna(how='any', subset = [response_variable])
#     print('Shape of data after excluding missing response:', np.shape(df))
    
    # delete columns with more than threshold NaN
    # get missing values < threshold feature name list
    missing_per = get_missing_val_percentage(df)     
    keep = df.columns[missing_per <= thresh]
    df = df[keep]
    print('Shape of data after removing cols with less than %.2f percent values missing:' % (thresh), df.shape)
   
    if not skip_corr:
        # remove correlated features 
        df = df.drop([x for x in correlated_variables if x in df.columns], axis=1)
        print('Shape of data after removing correlated features:', np.shape(df))

    
    # split data
    random.seed(42)
    # Save original data set
    original = df
    Y = df[response_variable]
    X = df.drop(response_variable, axis=1)
    if isms:
        splitter = GroupShuffleSplit(test_size=0.25, n_splits=1, random_state = 123)
        split = splitter.split(df, groups = df['patient.id'])
        train_inds, test_inds = next(split)

        train = df.iloc[train_inds]
        test = df.iloc[test_inds]

        groups =  np.array(train['patient.id'])
        X_train = train.drop([response_variable,'patient.id'], axis = 1)
        X_test = test.drop([response_variable, 'patient.id'], axis = 1)
        Y_train = train[response_variable]
        Y_test = test[response_variable]
    else:
        # Split into training and testing sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=123, stratify=Y)
    
    # data imputation
    original_X_train = X_train
    original_X_test = X_test
    
    imputer = SimpleImputer(missing_values=np.nan, strategy = "most_frequent")
    # imputeX = KNNImputer(missing_values=np.nan, n_neighbors = 3, weights='distance')
    # imputeX = IterativeImputer(max_iter=5, random_state=0)
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)
    
    # scale data
    scaler = MinMaxScaler()
    scaler2 = MinMaxScaler()
    
    select = {}
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train_imputed = scaler2.fit_transform(X_train_imputed)
    X_test_imputed = scaler2.transform(X_test_imputed)
        
    # print unique value count of response variable in train and test set
    unique_train, counts_train = numpy.unique(Y_train.to_numpy(), return_counts=True)
    unique_test, counts_test = numpy.unique(Y_test.to_numpy(), return_counts=True)
    print("Train - ", unique_train, counts_train)
    print("Test - ", unique_test, counts_test)
    
    X_train = pd.DataFrame(X_train, columns = original_X_train.columns, index=original_X_train.index)
    X_test = pd.DataFrame(X_test, columns = original_X_train.columns, index=original_X_test.index)
    
    X_train_imputed = pd.DataFrame(X_train_imputed, columns = original_X_train.columns, index=original_X_train.index)
    X_test_imputed = pd.DataFrame(X_test_imputed, columns = original_X_train.columns, index=original_X_test.index)
    
    return df, X_train, X_test, Y_train, Y_test, X, Y, X_train_imputed, X_test_imputed

In [ ]:
def cross_val(model, X_train, Y_train, n_splits=10):
    balacc_arr = []
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)

    for i, (train_index, test_index) in enumerate(kf.split(X_train, Y_train)):
        X_train1 = X_train.iloc[train_index]
        X_test1 = X_train.iloc[test_index]
        y_train1 = Y_train.iloc[train_index]
        y_test1 = Y_train.iloc[test_index]

        model.fit(X_train1, y_train1)

        # auc calculation
        y_scores = model.predict_proba(X_test1)

        # bal acc
        y_sc = y_scores[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test1, y_sc)
        optimal_idx = np.argmax(np.sqrt(tpr * (1 - fpr)))
        optimal_threshold = thresholds[optimal_idx]

        model_pred = (y_sc >= optimal_threshold).astype(int)
        confusion_matrix_model = confusion_matrix(y_test1, model_pred)
        acc_bal = confusion_matrix_model.diagonal() / confusion_matrix_model.sum(axis=1)
        balacc_arr.append(np.sum(acc_bal) / 2)

    variance_bal = np.var(balacc_arr, ddof=1)

    print("Cross validation variance Balance acc:", variance_bal)


    return model


In [ ]:
def plot_roc_curve(fpr, tpr, optimal_idx):
    plt.figure(figsize=(10,5))
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='black', label='Best') 
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
def optimal_thresh(model, X, Y):
    y_scores = model.predict_proba(X)[:,1]
    fpr, tpr, thresholds = roc_curve(Y, y_scores)
    optimal_idx = np.argmax(np.sqrt(tpr * (1-fpr)))
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

# def get_results(model, X, Y, pred):
#     confusion_matrix_model = confusion_matrix(Y, pred)
#     test_acc = model.score(X, Y)
    
#     results_table = [
#         ["Confusion Matrix", confusion_matrix_model],
#         ["Accuracy", test_acc]
#     ]
    
#     print(tabulate(results_table, headers=["Metric", "Value"], tablefmt="fancy_grid"))

def evaluate_model(model, X_test, Y_test, optimal_threshold):
    
    xgb_pred = (model.predict_proba(X_test)[:,1] >= optimal_threshold).astype(int)
    y_scores = model.predict_proba(X_test)
    score = round(roc_auc_score(Y_test, y_scores[:, 1]), 4)
    pred = model.predict(X_test)
    confusion_matrix_model = confusion_matrix(Y_test, xgb_pred)
    
    acc = confusion_matrix_model.diagonal() / confusion_matrix_model.sum(axis=1)
    avg_acc = np.sum(acc) / 2
    f1 = metrics.f1_score(xgb_pred, Y_test, average='micro')
    
    results_table = [
        ["Test ROC Score", score],
        ["Test Classwise Accuracy [Class 0, Class 1]", acc],
        ["Test Average Accuracy", avg_acc],
        ["Test F1 Score", f1]
    ]
    
    print(tabulate(results_table, headers=["Metric", "Value"], tablefmt="fancy_grid"))


def train_model(model, X_train, Y_train, X_test, Y_test):
    model = cross_val(model, X_train, Y_train)
#     model = model.fit(X_train, Y_train)
    optimal_threshold = optimal_thresh(model, X_test, Y_test)
    pred = (model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)
    optimal_threshold_train = optimal_thresh(model, X_train, Y_train)
    pred_train = (model.predict_proba(X_train)[:, 1] >= optimal_threshold_train).astype(int)

    optimal_threshold_test = optimal_thresh(model, X_test, Y_test)

    evaluate_model(model, X_test, Y_test, optimal_threshold_test)
    return model

In [ ]:
def init(models, X_train, X_test, X_train_imputed, X_test_imputed, Y_train, Y_test):
    trained_models = {}
    for model in models:
        X_t = X_train
        X_te = X_test
        model_name = model.__class__.__name__
        if(model_name == 'AdaBoostClassifier' or model_name == 'RandomForestClassifier' or model_name == 'LogisticRegression'):
            X_t = X_train_imputed
            X_te = X_test_imputed
        print("Model:", model_name)
        trained_model = train_model(model, X_t, Y_train, X_te, Y_test)
        trained_models[model_name] = trained_model
        print("\n" + "=" * 50 + "\n")
    return trained_models

In [ ]:

def get_important_features(best_n, X_train_feat, Y_train_feat):
    original_X = X_train
    fs = ReliefF()
    fs.fit(X_train_feat, Y_train_feat)

    feat_dict = {}

    for feature_name, feature_score in zip(original_X.columns,
                                           fs.feature_importances_):
        feat_dict[feature_name] = feature_score

    # sort and get most important features
    feat_names = []
    sorted_feat_dict = sorted(feat_dict.items(), key=lambda x: x[1], reverse=True)

    best = sorted_feat_dict[: best_n]

    for i in best:
        feat_names.append(i[0])

    return feat_names

In [ ]:
def remove_gender_specific_fields(df):
    drop_list = ['Pt_MenarcheAge','Pt_RegMenstCyc','Pt_IrregMenstCycReas','Pt_CurrPreg','Pt_MiscarriageNum']
    columns_to_drop_existing = [col for col in drop_list if col in df.columns]
    df = df.drop(columns=columns_to_drop_existing,  axis = 1)
    return df

In [ ]:

def get_gender_stratified_data(gender, train_, test_, train_imputed_, test_imputed_, Y_train_, Y_test_, variable):
    # remove female specific features from male dataser
    if gender == 0.5:
        train_ = remove_gender_specific_fields(train_)
        test_ = remove_gender_specific_fields(test_)
        train_imputed = remove_gender_specific_fields(train_imputed_)
        test_imputed = remove_gender_specific_fields(test_imputed_)
        
    # Update with proper value for females and males
    train_mask = train_[variable] == gender
    test_mask = test_[variable] == gender

    train_mask_imputed = train_imputed_[variable] == gender
    test_mask_imputed = test_imputed_[variable] == gender

    X_train = train_.loc[train_mask].drop(response_variable, axis=1)
    X_test = test_.loc[test_mask].drop(response_variable, axis=1)

    X_train_imputed_ = train_imputed_.loc[train_mask_imputed].drop(response_variable, axis=1)
    X_test_imputed_ = test_imputed_.loc[test_mask_imputed].drop(response_variable, axis=1)

    Y_train = Y_train_.loc[train_mask]
    Y_test = Y_test_.loc[test_mask]
   
    
    return X_train, X_train_imputed_, Y_train, X_test, X_test_imputed_, Y_test
